This notebook demonstrates how to use `pretrained-microscopy-models` to perform segmentation on ni-based superalloys. This is the Super-1 dataset in the accompanying paper.

In [8]:
import os
import torch
import cv2

import numpy as np
import pretrained_microscopy_models as pmm
import segmentation_models_pytorch as smp
import albumentations as albu

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

## Dataset

In [3]:
DATA_DIR = 'Super1/'

x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'train_annot')

x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'val_annot')

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'test_annot')

#### Image Augmentation
Adding noise, contrast, random crops, flipping and other image augmentations can artifically augment the training dataset and make the model more robust to changes in these conditions

In [6]:
def get_training_augmentation():
    train_transform = [
        albu.Flip(p=0.75),
        albu.RandomRotate90(p=1),       
        albu.IAAAdditiveGaussianNoise(p=0.5),
        
        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1, limit=0.25),
                albu.RandomGamma(p=1),
            ],
            p=0.50,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                #albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.50,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1, limit=0.3),
                albu.HueSaturationValue(p=1),
            ],
            p=0.50,
        ),
    ]
    return albu.Compose(train_transform)

def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        #albu.Resize(height,width)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [5]:
a = [1,2,3]
a = 1

In [6]:
type(a) is list

False

In [6]:
model = pmm.util.get_segmentation_model('Unet', 'resnet50', 'micronet', 3)

Downloading: "https://nasa-public-data.s3.amazonaws.com/microscopy_segmentation_models/resnet101_pretrained_microscopynet_v1.0.pth.tar" to C:\Users\jstuckne/.cache\torch\hub\checkpoints\resnet101_pretrained_microscopynet_v1.0.pth.tar
100%|██████████| 171M/171M [00:09<00:00, 19.2MB/s] 
